In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 52.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=25787b27df3de1cfdf7541020be7d991d1652b1a44e576dc510bb0dc6dae81cb
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()

In [5]:
df = spark.read.csv('/content/College.csv', inferSchema=True, header=True)

In [6]:
df.printSchema()
df.show(10)

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|              School|Private|Apps|Accept|E

In [8]:
from pyspark.ml.feature import VectorAssembler
df.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [9]:
assembler = VectorAssembler(inputCols=['Apps',
    'Accept',
    'Enroll',
    'Top10perc',
    'Top25perc',
    'F_Undergrad',
    'P_Undergrad',
    'Outstate',
    'Room_Board',
    'Books',
    'Personal',
    'PhD',
    'Terminal',
    'S_F_Ratio',
    'perc_alumni',
    'Expend',
    'Grad_Rate'], outputCol='features')

In [14]:
from pyspark.ml.feature import StringIndexer

output = assembler.transform(df)
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')
output_fixed = indexer.fit(output).transform(output)
output_fixed.show(5)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+------------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|            features|PrivateIndex|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+------------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|[1660.0,1232.0,72...|         0.0|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30

In [15]:
final_data = output_fixed.select(['features','PrivateIndex'])

In [16]:
train, test = final_data.randomSplit([0.7,0.3])

In [18]:
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier
from pyspark.ml import Pipeline

dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [19]:
dtc_model = dtc.fit(train)
rfc_model = rfc.fit(train)
gbt_model = gbt.fit(train)

dtc_preds = dtc_model.transform(test)
rfc_preds = rfc_model.transform(test)
gbt_preds = gbt_model.transform(test)

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

bin_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')
print(f"DTC: {bin_eval.evaluate(dtc_preds)}")
print(f"RFC: {bin_eval.evaluate(rfc_preds)}")
print(f"GBT: {bin_eval.evaluate(gbt_preds)}")

DTC: 0.9489087301587302
RFC: 0.9634176587301586
GBT: 0.947007275132275


In [23]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex', metricName='accuracy')
print(f"DTC accuracy: {acc_eval.evaluate(dtc_preds)}")
print(f"RFC accuracy: {acc_eval.evaluate(rfc_preds)}")
print(f"GBT accuracy: {acc_eval.evaluate(gbt_preds)}")

DTC accuracy: 0.8980392156862745
RFC accuracy: 0.9215686274509803
GBT accuracy: 0.9019607843137255
